In [ ]:

a = torch.arange(1, 17)  # a's shape is (16,)
 
a.view(-1,2,2) # output below

In [ ]:
tensor([[32004, 32005,   474,  1303, 10081,  8277,   263,  1629, 29889, 32000,
           474, 29915, 29885,   263,   380,  1657,  3765,   408,   590,  1473,
          4982, 29889, 32000,   474,   871, 17545,   413,   359,  2276, 29889,
         32000,   474,   471, 10425,   297,   263,  2323,  3847, 22329, 29889,
         32000, 32002, 22172,   825,   526,  2599,  9826,  1577,     2]],
       device='cuda:0')
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]], device='cuda:0')

tensor([[50267, 50268,   939,   524, 34574,    11, 47510,  2292, 28627,     8,
         41259,     4,     2,   127,  6578,     8,   939,    32,  1375,    88,
            41,  3537,   561,   220,   186,     4,     2,   127,  2674,  8089,
            32,  1275,     8,  4334,     4,     2,   939,   524,  2509,    11,
         11075,     8,   101,   602,  3493,     4,     2,   939,   524,    41,
         10891,   334,  3254,     4,     2, 50265,   141,    32,    47,   608,
           452, 17487,     2]], device='cuda:0')
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')

In [2]:
from torch.nn.utils.rnn import pad_sequence
import torch
import numpy as np
import transformers 
from transformers import AutoTokenizer
from transformers import LlamaForCausalLM
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
def prepare_llama_tokenizer_and_embedding(
        tokenizer: transformers.PreTrainedTokenizer,
        model: transformers.PreTrainedModel,
        special_tokens_dict: Dict = dict(pad_token=DEFAULT_PAD_TOKEN),
):
    """prepare llama tokenizer and embedding.

    """

    if tokenizer.pad_token is None:
        smart_tokenizer_and_embedding_resize(
            special_tokens_dict=dict(pad_token=DEFAULT_PAD_TOKEN),
            tokenizer=tokenizer,
            model=model,
        )

    tokenizer.add_special_tokens({
        "eos_token": DEFAULT_EOS_TOKEN,
        "bos_token": DEFAULT_BOS_TOKEN,
        "unk_token": DEFAULT_UNK_TOKEN,
    })

    return tokenizer

In [ ]:
item_mine = pad_sequence([torch.from_numpy(np.array(x)) for x in item_mine],
                                  batch_first=True, padding_value=pad_id)

In [3]:
#tokenizer = AutoTokenizer.from_pretrained("/checkpoint/llama_7B/", padding_side="right", use_fast=False)
tokenizer = AutoTokenizer.from_pretrained(
    "./checkpoint/llama_7B",
    padding_side="right",
    use_fast=False,
)
#model = LlamaForCausalLM.from_pretrained("./checkpoint/llama_7B")

In [ ]:
tokenizer.eos_token = '<\s>'
tokenizer.bos_token = '<s>'
special_tokens_dict = {"sep_token": "<\sep>", "pad_token": "<\pad>"}
tokenizer.add_special_tokens(special_tokens_dict)
tokenizer.add_tokens(['<query>', '<response>', '<latent>', '<persona>'])

In [ ]:
print(tokenizer.bos_token_id, tokenizer.eos_token_id, tokenizer.pad_token_id, tokenizer.sep_token_id)
print(tokenizer.bos_token, tokenizer.eos_token, tokenizer.pad_token, tokenizer.sep_token)
print(tokenizer.convert_tokens_to_ids(['<query>', '<response>', '<latent>', '<persona>']))

In [ ]:
tokenizer.sep_token='<\s>'

In [ ]:
tokenizer.eos_token = '<\s>'

In [7]:
print(tokenizer.sep_token_id)

None


In [ ]:
prompt = '<\s>'
inputs = tokenizer(prompt, return_tensors="pt")
print(inputs)

In [ ]:
generate_ids = model.generate(inputs.input_ids, max_length=30)
print(generate_ids)

In [ ]:
tokenizer.decode(inputs['input_ids'][0])

In [ ]:
print(tokenizer.decode(['hello']))

In [ ]:
print(len(tokenizer.get_vocab()))

In [ ]:
a = tokenizer.get_vocab()

In [ ]:
a = {v : k for k, v in a.items()}

In [ ]:
print(a[50])

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2DoubleHeadsModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2DoubleHeadsModel.from_pretrained('gpt2')

choices = [ "Bob likes candy ; what does Bob like ?  Bag <|endoftext|>",
                   "Bob likes candy ; what does Bob like ?  Burger <|endoftext|>",
                   "Bob likes candy ; what does Bob like ?  Candy <|endoftext|>",
                  "Bob likes candy ; what does Bob like ?  Apple <|endoftext|>"]

encoded_choices = [tokenizer.encode(s) for s in choices]

eos_token_location = [tokens.index(tokenizer.eos_token_id) for tokens in encoded_choices]
input_ids = torch.tensor(encoded_choices).unsqueeze(0) 
mc_token_ids = torch.tensor([eos_token_location]) 
print(input_ids.shape)
print(mc_token_ids.shape)
outputs = model(input_ids, mc_token_ids=mc_token_ids)
lm_prediction_scores, mc_prediction_scores = outputs[:2]


In [ ]:
print(lm_prediction_scores.shape)
print(mc_prediction_scores.shape)

In [ ]:
import wandb, random
total_step_num = 1000
for step in range(total_step_num):
    wandb.log({'random_curve':step/100+random.random()},step=step)
    wandb.log({'log_curve': math.log(step+1)},step=step)
wandb.finish()

In [ ]:
wandb.login()

In [ ]:
import random

# Launch 5 simulated experiments
total_runs = 5
for run in range(total_runs):
  # 🐝 1️⃣ Start a new run to track this script
  wandb.init(
      # Set the project where this run will be logged
      project="wandbexample1", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment_{run}", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.02,
      "architecture": "CNN",
      "dataset": "CIFAR-100",
      "epochs": 10,
      })
  
  # This simple block simulates a training loop logging metrics
  epochs = 10
  offset = random.random() / 5
  for epoch in range(2, epochs):
      acc = 1 - 2 ** -epoch - random.random() / epoch - offset
      loss = 2 ** -epoch + random.random() / epoch + offset
      
      # 🐝 2️⃣ Log metrics from your script to W&B
      wandb.log({"acc": acc, "loss": loss})
      
  # Mark the run as finished
  wandb.finish()



In [ ]:
['model.layers.0.self_attn.o_proj.lora_B', 
 'model.layers.24.mlp.up_proj.lora_B', 
 'model.layers.12.mlp.down_proj.lora_B', 
 'model.layers.8.self_attn.v_proj.lora_A', 
 'model.layers.12.self_attn.q_proj.lora_B', 
 'model.layers.21.mlp.up_proj.lora_B', 
 'model.layers.20.mlp.gate_proj.lora_B', 'model.layers.17.mlp.gate_proj.lora_B', 'model.layers.7.mlp.down_proj.lora_A', 'model.layers.14.mlp.up_proj.lora_A', 'lm_head.lora_A', 'model.layers.28.self_attn.o_proj.lora_A', 'model.layers.19.self_attn.v_proj.lora_A', 'model.layers.8.mlp.up_proj.lora_B', 'model.layers.31.mlp.gate_proj.lora_A', 'model.layers.23.mlp.down_proj.lora_A', 'model.layers.17.self_attn.k_proj.lora_A', 'model.layers.19.self_attn.q_proj.lora_A', 'model.layers.0.mlp.up_proj.lora_B', 'model.layers.15.self_attn.v_proj.lora_B', 'model.layers.5.self_attn.q_proj.lora_B', 'model.layers.20.mlp.up_proj.lora_A', 'model.layers.26.mlp.up_proj.lora_B', 'model.layers.25.self_attn.v_proj.lora_B', 'model.layers.6.mlp.gate_proj.lora_B', 'model.layers.21.self_attn.q_proj.lora_A', 'model.layers.9.self_attn.v_proj.lora_A', 'model.layers.16.mlp.down_proj.lora_A', 'model.layers.17.self_attn.q_proj.lora_B', 'model.layers.0.self_attn.k_proj.lora_A', 'model.layers.4.mlp.down_proj.lora_A', 'model.layers.22.mlp.down_proj.lora_A', 'model.layers.24.mlp.up_proj.lora_A', 'model.layers.3.mlp.down_proj.lora_B', 'model.layers.25.mlp.up_proj.lora_B', 'model.layers.3.self_attn.k_proj.lora_B', 'model.layers.13.mlp.gate_proj.lora_B', 'model.layers.15.mlp.gate_proj.lora_A', 'model.layers.22.self_attn.o_proj.lora_B', 'model.layers.27.self_attn.q_proj.lora_A', 'model.layers.1.self_attn.k_proj.lora_B', 'model.layers.27.mlp.down_proj.lora_A', 'model.layers.30.self_attn.q_proj.lora_A', 'model.layers.20.self_attn.v_proj.lora_A', 'model.layers.13.self_attn.q_proj.lora_A', 'model.layers.30.self_attn.k_proj.lora_A', 'model.layers.9.self_attn.k_proj.lora_B', 'model.layers.17.self_attn.v_proj.lora_A', 'model.layers.17.self_attn.o_proj.lora_B', 'model.layers.24.self_attn.o_proj.lora_A', 'model.layers.31.mlp.gate_proj.lora_B', 'model.layers.1.self_attn.q_proj.lora_A', 'model.layers.24.mlp.down_proj.lora_A', 'model.layers.7.mlp.gate_proj.lora_B', 'model.layers.15.self_attn.k_proj.lora_B', 'model.layers.2.self_attn.k_proj.lora_A', 'model.layers.6.self_attn.k_proj.lora_A', 'model.layers.27.self_attn.o_proj.lora_B', 'model.layers.27.self_attn.v_proj.lora_B', 'model.layers.29.self_attn.k_proj.lora_B', 'model.layers.25.mlp.down_proj.lora_A', 'model.layers.2.mlp.down_proj.lora_A', 'model.layers.14.self_attn.v_proj.lora_B', 'model.layers.29.self_attn.v_proj.lora_A', 'model.layers.26.self_attn.o_proj.lora_A', 'model.layers.25.self_attn.o_proj.lora_B', 'model.layers.14.mlp.down_proj.lora_A', 'model.layers.20.self_attn.k_proj.lora_A', 'model.layers.27.self_attn.q_proj.lora_B', 'model.layers.31.mlp.up_proj.lora_A', 'model.layers.22.mlp.down_proj.lora_B', 'model.layers.7.self_attn.o_proj.lora_A', 'model.layers.8.self_attn.o_proj.lora_B', 'model.layers.2.self_attn.k_proj.lora_B', 'model.layers.15.self_attn.q_proj.lora_B', 'model.layers.23.self_attn.q_proj.lora_A', 'model.layers.3.mlp.up_proj.lora_B', 'model.layers.7.self_attn.k_proj.lora_B', 'model.layers.5.self_attn.v_proj.lora_B', 'model.layers.20.self_attn.q_proj.lora_B', 'model.layers.7.self_attn.v_proj.lora_B', 'model.layers.11.self_attn.o_proj.lora_B', 'model.layers.31.mlp.up_proj.lora_B', 'model.layers.15.self_attn.k_proj.lora_A', 'model.layers.22.mlp.gate_proj.lora_B', 'model.layers.16.self_attn.q_proj.lora_A', 'model.layers.4.mlp.gate_proj.lora_B', 'model.layers.11.self_attn.v_proj.lora_A', 'model.layers.19.mlp.gate_proj.lora_A', 'model.layers.3.self_attn.k_proj.lora_A', 'model.layers.0.mlp.gate_proj.lora_B', 'model.layers.0.mlp.down_proj.lora_B', 'model.layers.17.self_attn.k_proj.lora_B', 'model.layers.29.self_attn.k_proj.lora_A', 'model.layers.6.self_attn.k_proj.lora_B', 'model.layers.26.mlp.up_proj.lora_A', 'model.layers.12.self_attn.k_proj.lora_B', 'model.layers.7.self_attn.q_proj.lora_A', 'model.layers.4.mlp.down_proj.lora_B', 'model.layers.18.self_attn.o_proj.lora_B', 'model.layers.24.self_attn.o_proj.lora_B', 'model.layers.10.self_attn.o_proj.lora_A', 'model.layers.22.self_attn.q_proj.lora_B', 'model.layers.27.self_attn.v_proj.lora_A', 'model.layers.5.self_attn.v_proj.lora_A', 'model.layers.24.mlp.gate_proj.lora_B', 'model.layers.15.mlp.up_proj.lora_B', 'model.layers.26.self_attn.v_proj.lora_B', 'model.layers.23.mlp.up_proj.lora_B', 'model.layers.30.mlp.up_proj.lora_A', 'model.layers.4.self_attn.q_proj.lora_B', 'model.layers.8.mlp.gate_proj.lora_A', 'model.layers.21.mlp.gate_proj.lora_B', 'model.layers.22.self_attn.v_proj.lora_B', 'model.layers.23.self_attn.q_proj.lora_B', 'model.layers.11.mlp.gate_proj.lora_B', 'model.layers.25.mlp.gate_proj.lora_A', 'model.layers.6.self_attn.q_proj.lora_A', 'model.layers.6.mlp.up_proj.lora_A', 'model.layers.6.mlp.down_proj.lora_B', 'model.layers.8.self_attn.k_proj.lora_B', 'model.layers.21.self_attn.o_proj.lora_A', 'model.layers.23.self_attn.o_proj.lora_A', 'model.layers.9.mlp.down_proj.lora_A', 'model.layers.7.mlp.gate_proj.lora_A', 'model.layers.26.self_attn.k_proj.lora_A', 'model.layers.7.self_attn.q_proj.lora_B', 'model.layers.0.mlp.up_proj.lora_A', 'model.layers.28.self_attn.v_proj.lora_B', 'model.layers.18.mlp.down_proj.lora_A', 'model.layers.11.mlp.up_proj.lora_A', 'model.layers.12.self_attn.o_proj.lora_A', 'model.layers.2.mlp.up_proj.lora_B', 'model.layers.1.mlp.up_proj.lora_B', 'model.layers.14.self_attn.q_proj.lora_A', 'model.layers.30.mlp.down_proj.lora_A', 'model.layers.13.mlp.gate_proj.lora_A', 'model.layers.5.mlp.down_proj.lora_B', 'model.layers.26.mlp.down_proj.lora_B', 'model.layers.23.self_attn.v_proj.lora_B', 'model.layers.22.mlp.up_proj.lora_B', 'model.layers.24.self_attn.k_proj.lora_A', 'model.layers.16.self_attn.q_proj.lora_B', 'model.layers.2.self_attn.v_proj.lora_B', 'model.layers.28.mlp.up_proj.lora_A', 'model.layers.3.self_attn.q_proj.lora_B', 'model.layers.25.self_attn.k_proj.lora_B', 'model.layers.14.mlp.up_proj.lora_B', 'model.layers.25.mlp.up_proj.lora_A', 'model.layers.8.mlp.down_proj.lora_B', 'model.layers.16.self_attn.k_proj.lora_B', 'model.layers.23.self_attn.k_proj.lora_B', 'model.layers.31.self_attn.o_proj.lora_B', 'model.layers.15.mlp.gate_proj.lora_B', 'model.layers.1.mlp.gate_proj.lora_B', 'model.layers.23.self_attn.v_proj.lora_A', 'model.layers.12.mlp.gate_proj.lora_A', 'model.layers.28.mlp.gate_proj.lora_A', 'model.layers.9.mlp.down_proj.lora_B', 'model.layers.19.self_attn.q_proj.lora_B', 'model.layers.29.self_attn.q_proj.lora_A', 'model.layers.25.mlp.gate_proj.lora_B', 'model.layers.20.mlp.up_proj.lora_B', 'model.layers.3.self_attn.o_proj.lora_A', 'model.layers.29.mlp.gate_proj.lora_B', 'model.layers.26.self_attn.o_proj.lora_B', 'model.layers.3.self_attn.o_proj.lora_B', 'model.layers.6.mlp.down_proj.lora_A', 'model.layers.7.mlp.down_proj.lora_B', 'model.layers.28.mlp.down_proj.lora_A', 'model.layers.11.self_attn.k_proj.lora_A', 'model.layers.15.mlp.down_proj.lora_B', 'model.layers.23.mlp.gate_proj.lora_B', 'model.layers.2.self_attn.q_proj.lora_B', 'model.layers.4.mlp.up_proj.lora_A', 'model.layers.21.self_attn.k_proj.lora_A', 'model.layers.16.self_attn.v_proj.lora_B', 'model.layers.31.self_attn.q_proj.lora_B', 'model.layers.16.mlp.down_proj.lora_B', 'model.layers.13.mlp.up_proj.lora_B', 'model.layers.24.self_attn.v_proj.lora_A', 'model.layers.1.mlp.gate_proj.lora_A', 'model.layers.15.self_attn.v_proj.lora_A', 'model.layers.13.mlp.up_proj.lora_A', 'model.layers.16.mlp.up_proj.lora_B', 'model.layers.20.mlp.down_proj.lora_A', 'model.layers.27.mlp.down_proj.lora_B', 'model.layers.11.self_attn.v_proj.lora_B', 'model.layers.16.mlp.gate_proj.lora_A', 'model.layers.25.self_attn.q_proj.lora_B', 'model.layers.15.self_attn.o_proj.lora_B', 'model.layers.12.mlp.up_proj.lora_B', 'model.layers.21.self_attn.k_proj.lora_B', 'model.layers.8.mlp.up_proj.lora_A', 'model.layers.5.mlp.gate_proj.lora_A', 'model.layers.17.self_attn.o_proj.lora_A', 'model.layers.31.mlp.down_proj.lora_A', 'model.layers.25.self_attn.v_proj.lora_A', 'model.layers.15.mlp.down_proj.lora_A', 'model.layers.21.self_attn.v_proj.lora_A', 'model.layers.27.mlp.up_proj.lora_A', 'model.layers.14.self_attn.o_proj.lora_B', 'model.layers.10.self_attn.q_proj.lora_B', 'model.layers.14.self_attn.q_proj.lora_B', 'model.layers.13.self_attn.o_proj.lora_A', 'model.layers.7.self_attn.o_proj.lora_B', 'model.layers.11.mlp.up_proj.lora_B', 'model.layers.9.mlp.gate_proj.lora_B', 'model.layers.27.self_attn.k_proj.lora_B', 'model.layers.29.mlp.down_proj.lora_B', 'model.layers.13.self_attn.k_proj.lora_A', 'model.layers.30.mlp.down_proj.lora_B', 'model.layers.11.mlp.gate_proj.lora_A', 'model.layers.1.self_attn.o_proj.lora_A', 'model.layers.17.self_attn.q_proj.lora_A', 'model.layers.10.self_attn.v_proj.lora_A', 'model.layers.4.self_attn.q_proj.lora_A', 'model.layers.13.self_attn.q_proj.lora_B', 'model.layers.23.self_attn.o_proj.lora_B', 'model.layers.5.self_attn.o_proj.lora_B', 'model.layers.23.mlp.gate_proj.lora_A', 'model.layers.18.self_attn.k_proj.lora_A', 'model.layers.16.self_attn.k_proj.lora_A', 'model.layers.24.mlp.gate_proj.lora_A', 'model.layers.1.self_attn.o_proj.lora_B', 'model.layers.4.self_attn.v_proj.lora_B', 'model.layers.23.mlp.up_proj.lora_A', 'model.layers.30.mlp.gate_proj.lora_B', 'model.layers.1.mlp.down_proj.lora_B', 'model.layers.17.mlp.gate_proj.lora_A', 'model.layers.20.self_attn.q_proj.lora_A', 'model.layers.9.mlp.gate_proj.lora_A', 'model.layers.13.mlp.down_proj.lora_A', 'model.layers.2.self_attn.o_proj.lora_B', 'model.layers.21.self_attn.o_proj.lora_B', 'model.layers.4.self_attn.k_proj.lora_B', 'model.layers.25.mlp.down_proj.lora_B', 'model.layers.16.self_attn.o_proj.lora_A', 'model.layers.30.self_attn.q_proj.lora_B', 'model.layers.7.self_attn.v_proj.lora_A', 'model.layers.9.self_attn.q_proj.lora_B', 'model.layers.18.self_attn.v_proj.lora_B', 'model.layers.16.self_attn.v_proj.lora_A', 'model.layers.23.mlp.down_proj.lora_B', 'model.layers.11.self_attn.q_proj.lora_A', 'model.layers.1.mlp.up_proj.lora_A', 'model.layers.19.self_attn.o_proj.lora_B', 'model.layers.29.mlp.gate_proj.lora_A', 'model.layers.31.self_attn.k_proj.lora_B', 'model.layers.28.mlp.up_proj.lora_B', 'model.layers.25.self_attn.o_proj.lora_A', 'model.layers.7.mlp.up_proj.lora_B', 'model.layers.12.self_attn.v_proj.lora_B', 'model.layers.29.self_attn.q_proj.lora_B', 'model.layers.9.self_attn.v_proj.lora_B', 'model.layers.19.mlp.up_proj.lora_B', 'model.layers.10.mlp.up_proj.lora_A', 'model.layers.6.self_attn.v_proj.lora_A', 'model.layers.14.self_attn.o_proj.lora_A', 'model.layers.23.self_attn.k_proj.lora_A', 'model.layers.14.self_attn.v_proj.lora_A', 'model.layers.8.mlp.down_proj.lora_A', 'model.layers.2.mlp.gate_proj.lora_A', 'model.layers.13.mlp.down_proj.lora_B', 'model.layers.22.self_attn.q_proj.lora_A', 'model.layers.8.self_attn.q_proj.lora_A', 'model.layers.13.self_attn.o_proj.lora_B', 'model.layers.30.self_attn.o_proj.lora_A', 'model.layers.28.self_attn.q_proj.lora_A', 'model.layers.12.mlp.down_proj.lora_A', 'model.layers.9.self_attn.o_proj.lora_A', 'model.layers.31.self_attn.v_proj.lora_B', 'model.layers.31.self_attn.o_proj.lora_A', 'model.layers.1.self_attn.v_proj.lora_B', 'model.layers.20.mlp.gate_proj.lora_A', 'model.layers.6.self_attn.v_proj.lora_B', 'model.layers.17.mlp.down_proj.lora_A', 'model.layers.10.mlp.gate_proj.lora_A', 'model.layers.11.mlp.down_proj.lora_A', 'model.layers.3.self_attn.v_proj.lora_B', 'model.layers.11.self_attn.o_proj.lora_A', 'model.layers.20.mlp.down_proj.lora_B', 'model.layers.18.mlp.gate_proj.lora_B', 'model.layers.15.mlp.up_proj.lora_A', 'model.layers.31.self_attn.k_proj.lora_A', 'model.layers.10.self_attn.k_proj.lora_B', 'model.layers.30.mlp.gate_proj.lora_A', 'model.layers.25.self_attn.q_proj.lora_A', 'model.layers.3.self_attn.v_proj.lora_A', 'model.layers.18.mlp.up_proj.lora_B', 'model.layers.7.mlp.up_proj.lora_A', 'model.layers.19.self_attn.k_proj.lora_B', 'model.layers.20.self_attn.k_proj.lora_B', 'model.layers.8.mlp.gate_proj.lora_B', 'model.layers.19.mlp.down_proj.lora_B', 'model.layers.31.self_attn.v_proj.lora_A', 'model.layers.6.self_attn.o_proj.lora_A', 'model.layers.0.mlp.down_proj.lora_A', 'model.layers.19.mlp.up_proj.lora_A', 'model.layers.22.self_attn.v_proj.lora_A', 'model.layers.26.self_attn.q_proj.lora_B', 'model.layers.20.self_attn.o_proj.lora_B', 'model.layers.24.mlp.down_proj.lora_B', 'model.layers.25.self_attn.k_proj.lora_A', 'model.layers.26.self_attn.q_proj.lora_A', 'model.layers.21.mlp.down_proj.lora_A', 'model.layers.4.self_attn.o_proj.lora_B', 'model.layers.28.self_attn.k_proj.lora_B', 'model.layers.26.mlp.gate_proj.lora_A', 'model.layers.11.mlp.down_proj.lora_B', 'model.layers.10.self_attn.v_proj.lora_B', 'model.layers.3.self_attn.q_proj.lora_A', 'model.layers.2.self_attn.o_proj.lora_A', 'model.layers.10.self_attn.o_proj.lora_B', 'model.layers.26.mlp.down_proj.lora_A', 'model.layers.0.self_attn.v_proj.lora_B', 'model.layers.5.self_attn.k_proj.lora_B', 'model.layers.3.mlp.down_proj.lora_A', 'model.layers.22.self_attn.k_proj.lora_A', 'model.layers.29.mlp.down_proj.lora_A', 'model.layers.0.self_attn.q_proj.lora_A', 'model.layers.0.mlp.gate_proj.lora_A', 'model.layers.17.mlp.down_proj.lora_B', 'model.layers.2.self_attn.q_proj.lora_A', 'model.layers.9.mlp.up_proj.lora_B', 'model.layers.17.mlp.up_proj.lora_A', 'model.layers.2.mlp.up_proj.lora_A', 'model.layers.19.self_attn.v_proj.lora_B', 'model.layers.31.self_attn.q_proj.lora_A', 'model.layers.2.mlp.down_proj.lora_B', 'model.layers.15.self_attn.o_proj.lora_A', 'model.layers.1.mlp.down_proj.lora_A', 'model.layers.2.self_attn.v_proj.lora_A', 'model.layers.12.self_attn.o_proj.lora_B', 'model.layers.29.self_attn.o_proj.lora_A', 'model.layers.12.mlp.up_proj.lora_A', 'model.layers.19.mlp.gate_proj.lora_B', 'model.layers.8.self_attn.k_proj.lora_A', 'model.layers.27.mlp.gate_proj.lora_A', 'model.layers.29.mlp.up_proj.lora_B', 'model.layers.0.self_attn.q_proj.lora_B', 'model.layers.11.self_attn.q_proj.lora_B', 'model.layers.12.self_attn.k_proj.lora_A', 'model.layers.12.self_attn.v_proj.lora_A', 'model.layers.6.self_attn.q_proj.lora_B', 'model.layers.21.self_attn.v_proj.lora_B', 'model.layers.1.self_attn.k_proj.lora_A', 'model.layers.15.self_attn.q_proj.lora_A', 'model.layers.3.mlp.gate_proj.lora_A', 'model.layers.0.self_attn.o_proj.lora_A', 'model.layers.12.self_attn.q_proj.lora_A', 'model.layers.18.mlp.down_proj.lora_B', 'model.layers.16.mlp.up_proj.lora_A', 'model.layers.16.self_attn.o_proj.lora_B', 'model.layers.8.self_attn.o_proj.lora_A', 'model.layers.19.self_attn.o_proj.lora_A', 'model.layers.26.self_attn.k_proj.lora_B', 'model.layers.14.mlp.down_proj.lora_B', 'model.layers.31.mlp.down_proj.lora_B', 'model.layers.4.self_attn.o_proj.lora_A', 'model.layers.8.self_attn.q_proj.lora_B', 'model.layers.4.mlp.gate_proj.lora_A', 'model.layers.5.mlp.down_proj.lora_A', 'model.layers.9.self_attn.k_proj.lora_A', 'model.layers.5.mlp.up_proj.lora_B', 'model.layers.30.self_attn.o_proj.lora_B', 'model.layers.6.mlp.up_proj.lora_B', 'model.layers.24.self_attn.q_proj.lora_B', 'model.layers.28.mlp.gate_proj.lora_B', 'model.layers.8.self_attn.v_proj.lora_B', 'model.layers.22.mlp.up_proj.lora_A', 'model.layers.22.self_attn.o_proj.lora_A', 'model.layers.5.self_attn.k_proj.lora_A', 'model.layers.5.mlp.up_proj.lora_A', 'model.layers.24.self_attn.q_proj.lora_A', 'model.layers.28.mlp.down_proj.lora_B', 'model.layers.3.mlp.gate_proj.lora_B', 'model.layers.3.mlp.up_proj.lora_A', 'model.layers.2.mlp.gate_proj.lora_B', 'model.layers.18.self_attn.o_proj.lora_A', 'model.layers.28.self_attn.o_proj.lora_B', 'model.layers.18.mlp.up_proj.lora_A', 'model.layers.16.mlp.gate_proj.lora_B', 'model.layers.4.mlp.up_proj.lora_B', 'model.layers.18.self_attn.k_proj.lora_B', 'model.layers.28.self_attn.q_proj.lora_B', 'model.layers.5.self_attn.q_proj.lora_A', 'model.layers.10.mlp.down_proj.lora_B', 'model.layers.21.mlp.down_proj.lora_B', 'model.layers.24.self_attn.k_proj.lora_B', 'model.layers.27.self_attn.k_proj.lora_A', 'model.layers.13.self_attn.k_proj.lora_B', 'model.layers.10.mlp.down_proj.lora_A', 'model.layers.21.mlp.gate_proj.lora_A', 'model.layers.10.mlp.up_proj.lora_B', 'model.layers.14.self_attn.k_proj.lora_B', 'model.layers.27.mlp.gate_proj.lora_B', 'model.layers.6.self_attn.o_proj.lora_B', 'model.layers.1.self_attn.q_proj.lora_B', 'model.layers.29.self_attn.o_proj.lora_B', 'model.layers.29.mlp.up_proj.lora_A', 'model.layers.30.self_attn.k_proj.lora_B', 'model.layers.5.mlp.gate_proj.lora_B', 'lm_head.lora_B', 'model.layers.0.self_attn.k_proj.lora_B', 'model.layers.24.self_attn.v_proj.lora_B', 'model.layers.13.self_attn.v_proj.lora_B', 'model.layers.18.self_attn.q_proj.lora_B', 'model.layers.12.mlp.gate_proj.lora_B', 'model.layers.6.mlp.gate_proj.lora_A', 'model.layers.9.self_attn.q_proj.lora_A', 'model.layers.5.self_attn.o_proj.lora_A', 'model.layers.1.self_attn.v_proj.lora_A', 'model.layers.27.mlp.up_proj.lora_B', 'model.layers.11.self_attn.k_proj.lora_B', 'model.layers.28.self_attn.v_proj.lora_A', 'model.layers.30.self_attn.v_proj.lora_A', 'model.layers.4.self_attn.k_proj.lora_A', 'model.layers.0.self_attn.v_proj.lora_A', 'model.layers.18.self_attn.v_proj.lora_A', 'model.layers.18.self_attn.q_proj.lora_A', 'model.layers.30.self_attn.v_proj.lora_B', 'model.layers.30.mlp.up_proj.lora_B', 'model.layers.9.self_attn.o_proj.lora_B', 'model.layers.20.self_attn.v_proj.lora_B', 'model.layers.7.self_attn.k_proj.lora_A', 'model.layers.19.mlp.down_proj.lora_A', 'model.layers.18.mlp.gate_proj.lora_A', 'model.layers.26.mlp.gate_proj.lora_B', 'model.layers.17.self_attn.v_proj.lora_B', 'model.layers.4.self_attn.v_proj.lora_A', 'model.layers.28.self_attn.k_proj.lora_A', 'model.layers.14.mlp.gate_proj.lora_B', 'model.layers.21.mlp.up_proj.lora_A', 'model.layers.14.self_attn.k_proj.lora_A', 'model.layers.29.self_attn.v_proj.lora_B', 'model.layers.10.mlp.gate_proj.lora_B', 'model.layers.20.self_attn.o_proj.lora_A', 'model.layers.10.self_attn.q_proj.lora_A', 'model.layers.17.mlp.up_proj.lora_B', 'model.layers.26.self_attn.v_proj.lora_A', 'model.layers.13.self_attn.v_proj.lora_A', 'model.layers.27.self_attn.o_proj.lora_A', 'model.layers.22.self_attn.k_proj.lora_B', 'model.layers.21.self_attn.q_proj.lora_B', 'model.layers.9.mlp.up_proj.lora_A', 'model.layers.14.mlp.gate_proj.lora_A', 'model.layers.22.mlp.gate_proj.lora_A', 'model.layers.19.self_attn.k_proj.lora_A', 'model.layers.10.self_attn.k_proj.lora_A']

In [ ]:
 yes , i love fresh veggies . what do you do for a living ?
--------------------------------------------------
 i don't have any pets , but i do travel a lot . i go to europe twice a year .
--------------------------------------------------
 i travel to europe twice a year . do you have any family ?
--------------------------------------------------
 my weekends are great , i go to europe twice a year .
--------------------------------------------------
 i try to go at least once a year . do you have family ?
--------------------------------------------------
 i've been to australia twice , it is beautiful . do you have family there ?
--------------------------------------------------
 i do , i love to walk with my family . i am a descendant of christopher columbus .
--------------------------------------------------
274s elapsed: {'exs': 2194, '%done': '28.12%', 'time_left': '702s', 'f1': 0.2204, 'bleu': 0.01586}
 i am doing well . how are you ?
--------------------------------------------------

In [ ]:
--------------------------------------------------
4738s elapsed: {'exs': 397, '%done': '5.09%', 'time_left': '88367s', 'f1': 0.03384, 'bleu': 0.0006517}
<latent><persona> two dogs live with me. i've short hair. i like doing the treadmill and rowing machine. i go to the gym regularly.<query> hello , my name is leon . i am a audio engineer . how are you ?<response> hello , leon . my dogs and i are doing well .<query> that is good to hear ! i've been better , my girlfriend just left me .<response> oh no . i go to the gym a lot . the treadmill keeps my mind off things .<query> i tend to stick indoors but maybe the gym will help , thanks .<response> you are most welcome . the rowing machine is nice too .<query> that's a full body work out right there . do you live in california ? The 2018-19 school year is off to a great start! We are excited to welcome our new students and families to the school and to welcome back our returning students and families.
We are looking forward to a great year of learning and growing together.
Please take a moment to review the information below and the links to the left.
If you have any questions, please contact the school office at 780-467-2200.
The school office is open from 8:00 am to 4:00 pm.
The school office is closed for lunch from 12:00 pm to 12:30 pm.
The school office is closed for lunch from 12:00 pm to 12:30 pm.
The school office is closed for lunch from 12:00 pm to 12:30 pm.
The school office is closed for lunch from 12:00 pm to 12:30 pm.
The school office is closed for lunch from 12:00 pm to 12:30 pm.
The school office is closed for lunch from 12:00 pm to 12:30 pm.
The school office is closed for lunch from 12:00 pm to 12:30 pm.
The school office is closed for lunch from 12:00 pm to 12:30 pm.
The school office is closed for lunch
--------------------------------------------------
4749s elapsed: {'exs': 398, '%done': '5.10%', 'time_left': '88351s', 'f1': 0.03387, 'bleu': 0.0006797}
<latent><persona> two dogs live with me. i've short hair. i like doing the treadmill and rowing machine. i go to the gym regularly.<query> hello , my name is leon . i am a audio engineer . how are you ?<response> hello , leon . my dogs and i are doing well .<query> that is good to hear ! i've been better , my girlfriend just left me .<response> oh no . i go to the gym a lot . the treadmill keeps my mind off things .<query> i tend to stick indoors but maybe the gym will help , thanks .<response> you are most welcome . the rowing machine is nice too .<query> that's a full body work out right there . do you live in california ?<response> sadly , no . my dogs and i are in ohio .<query> i think i need to move . my studio is tanking after the decline of the market . The 2018-19 school year is off to a great start! We are excited to welcome our new students and families to the school and to welcome back our returning students and families.
We are looking forward to a great year of learning and growing together.
Please take a moment to review the information below and the links to the left.
The first day of school is Tuesday, September 4th.
The first day of school for students is Tuesday, September 4th.
The first day of school for students is Wednesday, September 5th.
The first day of school for students is Thursday, September 6th.
The first day of school for students is Friday, September 7th.
The first day of school for students is Monday, September 10th.
The first day of school for students is Tuesday, September 111th.
The first day of school for students is Wednesday, September 12th.
The first day of school for students is Thursday, September 13th.
The first day of school for students is Friday, September 14th.
The first day of school for students is Monday, September 17th.
The first day of school for students is Tuesday, September 18th.
The first day of school for students is Wednesday,
--------------------------------------------------
4760s elapsed: {'exs': 399, '%done': '5.11%', 'time_left': '88312s', 'f1': 0.03386, 'bleu': 0.000678}
<latent><persona> two dogs live with me. i've short hair. i like doing the treadmill and rowing machine. i go to the gym regularly.<query> hello , my name is leon . i am a audio engineer . how are you ?<response> hello , leon . my dogs and i are doing well .<query> that is good to hear ! i've been better , my girlfriend just left me .<response> oh no . i go to the gym a lot . the treadmill keeps my mind off things .<query> i tend to stick indoors but maybe the gym will help , thanks .<response> you are most welcome . the rowing machine is nice too .<query> that's a full body work out right there . do you live in california ?<response> sadly , no . my dogs and i are in ohio .<query> i think i need to move . my studio is tanking after the decline of the market .<response> oh no . i'm moving soon as well . due to personal reasons .<query> well i hope your reasons work themselves out . what kind of dogs do you have ? The 2018-19 school year is off to a great start! We are excited to welcome our new students and families to the school and to welcome back our returning students and families.
We are looking forward to a great year of learning and growing together.
Please take a moment to review the information below and the links to the left.
The first day of school is Tuesday, September 4th.
The first day of school for students is Tuesday, September 4th.
The first day of school for students is Tuesday, September 4th. The first day of school for teachers is Monday, September 3rd.
The first day of school for students is Tuesday, September 4th. The first day of school for teachers is Monday, September 3rd.
The first day of school for students is Tuesday, September 4th. The first day of school for teachers is Monday, September 3rd. The first day of school for teachers is Monday, September 3rd.
The first day of school for students is Tuesday, September 4th. The first day of school for teachers is Monday, September 3rd. The first day of school for teachers is Monday
--------------------------------------------------
4769s elapsed: {'exs': 400, '%done': '5.13%', 'time_left': '88251s', 'f1': 0.03386, 'bleu': 0.0006763}
<latent><persona> two dogs live with me. i've short hair. i like doing the treadmill and rowing machine. i go to the gym regularly.<query> hello , my name is leon . i am a audio engineer . how are you ?<response> hello , leon . my dogs and i are doing well .<query> that is good to hear ! i've been better , my girlfriend just left me .<response> oh no . i go to the gym a lot . the treadmill keeps my mind off things .<query> i tend to stick indoors but maybe the gym will help , thanks .<response> you are most welcome . the rowing machine is nice too .<query> that's a full body work out right there . do you live in california ?<response> sadly , no . my dogs and i are in ohio .<query> i think i need to move . my studio is tanking after the decline of the market .<response> oh no . i'm moving soon as well . due to personal reasons .<query> well i hope your reasons work themselves out . what kind of dogs do you have ?<response> i've a springer spaniel and a lab . do you have pets ?<query> sadly no , they can be very destructive in an audio booth . The 2018-19 school year is off to a great start! We are excited to welcome our new students and families to the school and to welcome back our returning students and families.
We are looking forward to a great year of learning and growing together.
Please take a moment to review the information below and the links to the left.
The first day of school is Tuesday, September 4th.
The first day of school for students is Tuesday, September 4th.
The first day of school for students is Tuesday, September 4th. The first day of school for students is Tuesday, September 4th.
The first day of school for students is Tuesday, September 4th. The first day of school for students is Tuesday, September 4th.
The first day of school for students is Tuesday, September 4th. The first day of school for students is Tuesday, September 4th. The first day of school for students is Tuesday, September
--------------------------------------------------

In [ ]:
{'input_ids': tensor([[  366,  1818,   367,  1407,  5172,   869, 29826,   338,   697,   310,
           590, 25448,   298, 20838,   583,   869,     2,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0],
        [  474,   884,  1083, 27224, 17774,   746,   474,   626,   451,   714,
         12580, 29826,   869,     2,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0]]), 'labels': tensor([[  366,  1818,   367,  1407,  5172,   869, 29826,   338,   697,   310,
           590, 25448,   298, 20838,   583,   869,     2,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0],
        [  474,   884,  1083, 27224, 17774,   746,   474,   626,   451,   714,
         12580, 29826,   869,     2,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0]])}

In [ ]:
 1577, 2, 32003, 4874, 1919, 474, 626, 515, 278, 1629, 29871, 29896, 29947, 29953, 29900, 869, 474, 471, 6296, 1244, 408, 263, 19532, 869, 2, 32002, 8031, 1738, 474, 29915, 29885, 6142, 263, 16336, 14497, 264, 322, 515, 393, 3152, 869, 738, 413, 4841, 1577, 2], [32004, 32005, 474, 471, 278, 937, 6345, 2278, 29889, 32000, 474, 19824, 322, 2678, 3897, 263, 1559, 29886, 5893, 29889, 32000, 474, 2355, 8300, 29871, 29896, 29900, 2440, 2678, 29889, 32000, 474, 471, 263, 19532, 363, 29871, 29896, 29900, 2440, 29889, 32000, 32002, 22172, 920, 526, 366, 445, 11005, 1577, 2, 32003, 22172, 1919, 474, 626, 2691, 6452, 366, 869, 474, 626, 925, 28321, 590, 521, 2361, 869, 2, 32002, 474, 626, 408, 1532, 869, 508, 366, 2649, 592, 263, 2586, 1048, 7535, 1577, 2, 32003, 4874, 1919, 474, 626, 515, 278, 1629, 29871, 29896, 29947, 29953, 29900, 869, 474, 471, 6296, 1244, 408, 263, 19532, 869, 2, 32002, 8031, 1738, 474, 29915, 29885, 6142, 263, 16336, 14497, 264, 322, 515, 393, 3152, 869, 738, 413, 4841, 1577, 2, 32003, 474, 29915, 29885, 8300, 304, 263, 10404, 19532, 322, 591, 750, 1023, 4344, 4208, 869, 2, 32002, 590, 10216, 471, 263, 6413, 899, 1241, 322, 591, 505, 29871, 29946, 413, 4841, 869, 2], [32004, 32005, 474, 471, 278, 937, 6345, 2278, 29889, 32000, 474, 19824, 322, 2678, 3897, 263, 1559, 29886, 5893, 29889, 32000, 474, 2355, 8300, 29871, 29896, 29900, 2440, 2678, 29889, 32000, 474, 471, 263, 19532, 363, 29871, 29896, 29900, 2440, 29889, 32000, 32002, 22172, 920, 526, 366, 445, 11005, 1577, 2, 32003, 22172, 1919, 474, 626, 2691, 6452, 366, 869, 474, 626, 925, 28321, 590, 521, 2361, 869, 2, 32002, 474, 626, 408, 1532, 869, 508, 366, 2649, 592, 263, 2586, 1048, 7535, 1577, 2, 32003, 4874, 1919, 474, 626, 515, 278, 1629, 29871, 29896, 29947, 29953, 29900, 869, 474, 471, 6296, 1244, 408, 263, 19532, 869, 2, 32002, 8031, 1738, 474, 29915, 29885, 6142, 263, 16336, 14497, 264, 322, 515, 393, 3152, 869, 738, 413, 4841, 1577, 2, 32003, 474, 29915, 29885, 8300, 304, 263, 10404, 19532, 322, 591, 750, 1023, 4344, 4208, 869, 2, 32002, 590, 10216, 471, 263, 6413, 899, 1241, 322, 591, 505, 29871, 29946, 413, 4841, 869, 2, 32003, 20695, 1738, 474, 881, 15544, 869, 590, 6532, 322, 474, 1716, 19824, 1156, 29871, 29896, 29900, 2440, 408, 25569, 869, 2, 32002, 393, 29915, 29879, 29811, 14981, 766, 354, 8109, 292, 304, 8293, 869, 2], [32004, 32005, 474, 471, 278, 937, 6345, 2278, 29889, 32000, 474, 19824, 322, 2678, 3897, 263, 1559, 29886, 5893, 29889, 32000, 474, 2355, 8300, 29871, 29896, 29900, 2440, 2678, 29889, 32000, 474, 471, 263, 19532, 363, 29871, 29896, 29900, 2440, 29889, 32000, 32002, 22172, 920, 526, 366, 445, 11005, 1577, 2, 32003, 22172, 1919, 474, 626, 2691, 6452, 366, 869, 474, 626, 925, 28321, 590, 521, 2361, 869, 2, 32002, 474, 626, 408, 1532, 869, 508, 366, 2649, 592, 263, 2586, 1048, 7535, 1577, 2, 32003, 4874, 1919, 474, 626, 515, 278, 1629, 29871, 29896, 29947, 29953, 29900, 869, 474, 471, 6296, 1244, 408, 263, 19532, 869, 2, 32002, 8031, 1738, 474, 29915, 29885, 6142, 263, 16336, 14497, 264, 322, 515, 393, 3152, 869, 738, 413, 4841, 1577, 2, 32003, 474, 29915, 29885, 8300, 304, 263, 10404, 19532, 322, 591, 750, 1023, 4344, 4208, 869, 2, 32002, 590, 10216, 471, 263, 6413, 899, 1241, 322, 591, 505, 29871, 29946, 413, 4841, 869, 2, 32003, 20695, 1738, 474, 881, 15544, 869, 590, 6532, 322, 474, 1716, 19824, 1156, 29871, 29896, 29900, 2440, 408, 25569, 869, 2, 32002, 393, 29915, 29879, 29811, 14981, 766, 354, 8109, 292, 304, 8293, 869, 2, 32003, 1532, 591, 19824, 322, 1539, 1449, 29871, 29896, 29900, 2440, 2678, 322, 892, 8300, 869, 591, 526, 9796, 869, 2, 32002, 393, 29915, 29879, 1781, 1738, 738, 4595, 29895, 4841, 3447, 1577, 2], [32004, 32005, 474, 471, 278, 937, 6345, 2278, 29889, 32000, 474, 19824, 322, 2678, 3897, 263, 1559, 29886, 5893, 29889, 32000, 474, 2355, 8300, 29871, 29896, 29900, 2440, 2678, 29889, 32000, 474, 471, 263, 19532, 363, 29871, 29896, 29900, 2440, 29889, 32000, 32002, 22172, 920, 526, 366, 445, 11005, 1577, 2, 32003, 22172, 1919, 474, 626, 2691, 6452, 366, 869, 474, 626, 925, 28321, 590, 521, 2361, 869, 2, 32002, 474, 626, 408, 1532, 869, 508, 366, 2649, 592, 263, 2586, 1048, 7535, 1577, 2, 32003, 4874, 1919, 474, 626, 515, 278, 1629, 29871, 29896, 29947, 29953, 29900, 869, 474, 471, 6296, 1244, 408, 263, 19532, 869, 2, 32002, 8031, 1738, 474, 29915, 29885, 6142, 263, 16336, 14497, 264, 322, 515, 393, 3152, 869, 738, 413, 4841, 1577, 2, 32003, 474, 29915, 29885, 8300, 304, 263, 10404, 19532, 322, 591, 750, 1023, 4344, 4208, 869, 2, 32002, 590, 10216, 471, 263, 6413, 899, 1241, 322, 591, 505, 29871, 29946, 413, 4841, 869, 2, 32003, 20695, 1738, 474, 881, 15544, 869, 590, 6532, 322, 474, 1716, 19824, 1156, 29871, 29896, 29900, 2440, 408, 25569, 869, 2, 32002, 393, 29915, 29879, 29811, 14981, 766, 354, 8109, 292, 304, 8293, 869, 2, 32003, 1532, 591, 19824, 322, 1539, 1449, 29871, 29896, 29900, 2440, 2678, 322, 892, 8300, 869, 591, 526, 9796, 869, 2, 32002, 393, 29915, 29879, 1781, 1738, 738, 4595, 29895, 4841, 3447, 1577, 2, 32003, 1716, 310, 1749, 4344, 750, 289, 370, 583, 869, 591, 505, 29871, 29945, 4595, 29881, 6334, 2153, 322, 1023, 22114, 787, 869, 2, 32002, 590, 10216, 322, 474, 29915, 345, 4595, 29895, 4841, 393, 3464, 515, 29871, 29955, 304, 29871, 29941, 29900, 869, 727, 526, 29871, 29896, 29900, 3001, 869, 2], [32004, 32005, 474, 471, 278, 937, 6345, 2278, 29889, 32000, 474, 19824, 322, 2678, 3897, 263, 1559, 29886, 5893, 29889, 32000, 474, 2355, 8300, 29871, 29896, 29900, 2440, 2678, 29889, 32000, 474, 471, 263, 19532, 363, 29871, 29896, 29900, 2440, 29889, 32000, 32002, 22172, 920, 526, 366, 445, 11005, 1577, 2, 32003, 22172, 1919, 474, 626, 2691, 6452, 366, 869, 474, 626, 925, 28321, 590, 521, 2361, 869, 2, 32002, 474, 626, 408, 1532, 869, 508, 366, 2649, 592, 263, 2586, 1048, 7535, 1577, 2, 32003, 4874, 1919, 474, 626, 515, 278, 1629, 29871, 29896, 29947, 29953, 29900, 869, 474, 471, 6296, 1244, 408, 263, 19532, 869, 2, 32002, 8031, 1738, 474, 29915, 29885, 6142, 263, 16336, 14497, 264, 322, 515, 393, 3152, 869, 738, 413, 4841, 1577, 2, 32003, 474, 29915, 29885, 8300, 304, 263, 10404, 19532, 322, 591, 750, 1023, 4344, 4208, 869, 2, 32002, 590, 10216, 471, 263, 6413, 899, 1241, 322, 591, 505, 29871, 29946, 413, 4841, 869, 2, 32003, 20695, 1738, 474, 881, 15544, 869, 590, 6532, 322, 474, 1716, 19824, 1156, 29871, 29896, 29900, 2440, 408, 25569, 869, 2, 32002, 393, 29915, 29879, 29811, 14981, 766, 354, 8109, 292, 304, 8293, 869, 2, 32003, 1532, 591, 19824, 322, 1539, 1449, 29871, 29896, 29900, 2440, 2678, 322, 892, 8300, 869, 591, 526, 9796, 869, 2, 32002, 393, 29915, 29879, 1781, 1738, 738, 4595, 29895, 4841, 3447, 1577, 2, 32003, 1716, 310, 1749, 4344, 750, 289, 370, 583, 869, 591, 505, 29871, 29945, 4595, 29881, 6334, 2153, 322, 1023, 22114, 787, 869, 2, 32002, 590, 10216, 322, 474, 29915, 345, 4595, 29895, 4841, 393, 3464, 515, 29871, 29955, 304, 29871, 29941, 29900, 869, 727, 526, 29871, 29896, 29900, 3001, 869, 2, 32003, 4595, 11991, 526, 263, 17065, 292, 1738, 474, 1010, 366, 13389, 963, 1738, 2, 32002, 896, 1854, 526, 869, 591, 437, 1738, 727, 526, 29871, 29947, 14000, 322, 29871, 29906, 12544, 869, 2]]